In [1]:
#start

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1727667530908_0002,pyspark,idle,Link,Link,user_n-renukumar_rajendran_draup_com,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.addPyFile("s3://jobs-emr-etl-config/jobs_run/prod/cc/ML_ETL_Config.py")
from ML_ETL_Config import *
utils_module = MlEtlUtils(spark, sc)
to_import = [name for name in dir(utils_module) if not (name.startswith('_') | (name=="spark") | (name=="sc"))]
globals().update({name: getattr(utils_module, name) for name in to_import})
from pyspark.sql.functions import *
from pyspark.sql import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

404
404
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

In [3]:
dpd=spark.read.format('delta').load("s3://etl-datahouse/prod/profiles/dpd/delta").select('currentPositions','pastPositions','publicLink', 'fullName', 'gender','ethnicity','functional_workload','business_function','skills','msa','country','industry').selectExpr('*','currentPositions[0].companyName as curr_company','currentPositions[0].job_title_lemmatized as current_job_title','currentPositions[0].msa as current_msa','currentPositions[0].synon_location as current_synon_location','currentPositions[0].i18nStartDate as current_StartDate','currentPositions[0].i18nEndDate as current_EndDate','currentPositions[0].startDateYear as startDateYear','currentPositions[0].endDateYear as endDateYear','currentPositions.summary[0] as curr_summary','pastPositions[0].companyName as past_company','pastPositions[0].i18nStartDate as past_startDateYear','pastPositions[0].i18nEndDate as past_endDateYear','pastPositions[0].job_title_lemmatized as past_title','pastPositions[0].msa as past_msa','pastPositions[0].synon_location as past_synon_location').withColumn('lower_curr_company',F.lower(col('curr_company'))).withColumn('lower_past_company',F.lower(col('past_company'))).withColumn('past_data',col('pastPositions').getItem(0)).drop('currentPositions','pastPositions','past_data')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
dpd = spark.read.parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/temp/dpd_profile_data_27-09-2024')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
dpd.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- publicLink: string (nullable = true)
 |-- fullName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ethnicity: struct (nullable = true)
 |    |-- category: string (nullable = true)
 |    |-- probability: double (nullable = true)
 |-- functional_workload: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- business_function: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- msa: string (nullable = true)
 |-- country: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- curr_company: string (nullable = true)
 |-- current_job_title: string (nullable = true)
 |-- current_msa: string (nullable = true)
 |-- current_synon_location: string (nullable = true)
 |-- current_StartDate: string (nullable = true)
 |-- current_EndDate: string (nullable = true)
 |-- startDateYear: string (nullable = true)
 |-- endDateYear:

### Input File

In [19]:
df = spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/keyword_samplie_input_30-09-24.csv",header = True).withColumn('companyName_lower', lower(col('Company Name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Company Name', 'Keywords', 'companyName_lower']

In [21]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

49

In [22]:
result_df = dpd.crossJoin(df).filter(col('curr_summary').contains(col('Keywords'))).drop('past_company', 'past_startDateYear', 'past_endDateYear', 'past_title', 'past_msa', 'past_synon_location', 'lower_curr_company', 'lower_past_company','companyName_lower')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
result_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

408079

In [9]:
result_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['publicLink', 'fullName', 'gender', 'ethnicity', 'functional_workload', 'business_function', 'skills', 'msa', 'country', 'industry', 'curr_company', 'current_job_title', 'current_msa', 'current_synon_location', 'current_StartDate', 'current_EndDate', 'startDateYear', 'endDateYear', 'curr_summary', 'Company Name', 'Keywords']

In [10]:
split_ratios = [1/4] * 4  # Equal split ratios for 9 parts
split_datasets = result_df.randomSplit(split_ratios, seed=42)

# Access each split DataFrame
split_df1 = split_datasets[0]
split_df2 = split_datasets[1]
split_df3 = split_datasets[2]
split_df4 = split_datasets[3]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=result_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/dpd_profile_summary_data_set1_30-09-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '709BR37CFYMRAXDF', 'HostId': 'NzbO7zXRdcWnpN978r0WM0HuuXqpnN+jvY+xHh36RTpSPLlVm9W2jnbfO3vSDVBMFmKDJOvBkZhglNXkZVLtmQ==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'NzbO7zXRdcWnpN978r0WM0HuuXqpnN+jvY+xHh36RTpSPLlVm9W2jnbfO3vSDVBMFmKDJOvBkZhglNXkZVLtmQ==', 'x-amz-request-id': '709BR37CFYMRAXDF', 'date': 'Mon, 30 Sep 2024 04:31:31 GMT', 'x-amz-expiration': 'expiry-date="Sun, 06 Oct 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"baf8ec0027cb795486d9471c1f3e8a87"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Sun, 06 Oct 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"baf8ec0027cb795486d9471c1f3e8a87"', 'ServerSideEncryption': 'AES256'}

In [15]:
com_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['publicLink', 'fullName', 'gender', 'ethnicity', 'functional_workload', 'business_function', 'skills', 'msa', 'country', 'industry', 'curr_company', 'current_job_title', 'current_msa', 'current_synon_location', 'current_StartDate', 'current_EndDate', 'startDateYear', 'endDateYear', 'curr_summary', 'past_company', 'past_startDateYear', 'past_endDateYear', 'past_title', 'past_msa', 'past_synon_location', 'lower_curr_company', 'lower_past_company', 'Company Name', 'Keywords', 'companyName_lower']